# 감성 분석의 개념
감성 분석(sentiment analysis)이란 텍스트에 나타난 필자의 감성을 분석하는 과제이다. 그런데 **감성**을 정의한다는 것이 간단한 일이 아니기에 매우 다양한 논의 거리가 존재한다. 감성은 감정, 혹은 느낌과는 어떻게 다른지, 또한 다른 주관적 판단과는 어떻게 다른지는 아직 정리되어 있지 않다. 이 강좌에서는 이에 관한 논의는 생략한다.

한편 감성 분석은 오피니언 마이닝이라는 용어로도 쓰인다. 보통 이 과제는 텍스트에 나타난 표현이 "긍정적"인지 아니면 "부정적"인지를 판단하는 일로 주어진다. 물론 무엇이 긍정적이고 부정적인 것인지는 여전히 논란이 남지만, 어떤 대상에 대한 "평가"로 분석 대상을 좁히면 앞서 살펴본 문서 분류 과제가 된다.

위와는 성격이 다른 감성 분석의 예로 글에 나타난 "기분"을 분석하는 경우가 있다. 이 경우에는 인간이 느끼는 구체적인 감성, 혹은 감정(기쁨, 슬픔 등)가 어떻게 나타났는가를 살펴야 한다. 텍스트의 속성을 구체적인 감정에 사상하는 일은 결코 쉬운 일이 아니다. 현재 주로 사용되는 기법은 미리 만들어진 감성어 사전을 이용하여 텍스트의 어휘 분포를 파악하고 이를 구체적인 감성의 분석에 이용하는 것이다.

# 문서 분류에 의한 감성 분석
네이버의 영화 리뷰 테스트 자료를 배포 사이트(<https://github.com/e9t/nsmc>)에서 내려받아 영화 리뷰에 대한 감성 분석을 해보자. 이 자료는 총 200,000 개의 140 자 이내 리뷰로 구성되어 있으며 4점 이하를 부정 평가로 9점 이상을 긍정 평가로 처리하였다. 원자료는 TSV 형식의 텍스트 파일인데, 이를 형태소 분석하여 JSON 라인 형식의 파일로 구성하였다.

>실제로는 64만 건의 리뷰에서 20만 건을 표본 추출한 것이다.

![영화 리뷰의 점수 분포](figs/ratingdist.png)

이제 위의 형태소 분석 결과를 입력으로 하여 감성 분석을 수행한다. 분류 알고리즘으로는 나이브 베이즈를 사용하고 앞 강의에서 그리드 검색을 통해 얻어진 파라미터들을 그대로 사용하며 성능 측정을 5절 교차 검증으로 한다. 또 하나의 특징은 동형이품사어의 구별을 위해 형태소와 품사를 '/' 문자로 결합하여 사용하는 것이다. 이를 유지하기 위해 사용자 정의 토크나이저를 이용해야 하므로 문자열의 `split()` 메소드를 토크나이저로 지정한다. 

In [4]:
import ujson
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold
#from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

FEATURE_POSES = ["NC", "NQ", "PV", "PA", "MM", "MA", "II", "S"]
MA_KEY = "document_ma"
LABEL_KEY = "label"
NUM_FOLDS = 5


def read_documents_with_labels(input_file_name):
    """주어진 이름을 파일에서 문서와 레이블들을 읽어서 돌려준다."""
    
    documents = []
    labels = []

    with open(input_file_name, "r", encoding="utf-8") as input_file:
        for line in input_file:
            morphs = []
            json_obj = ujson.loads(line)
            label = json_obj[LABEL_KEY]
            labels.append(label)

            for sent_anal in json_obj[MA_KEY]:
                for morph_lex, morph_cat in sent_anal:
                    if morph_cat not in FEATURE_POSES:
                        continue

                    # 형태소 어형과 품사를 하나의 문자열로 결합하여 사용한다.
                    morphs.append(morph_lex + "/" + morph_cat)

            document = " ".join(morphs)
            documents.append(document)

    # scikit의 다양한 기능을 이용하기 위해서
    # 파이썬 리스트를 numpy 모듈의 배열(array)로 바꾼다.
    documents = np.asarray(documents)
    labels = np.asarray(labels)
    
    return documents, labels


def build_pipeline():
    """벡터라이저와 분류자의 파이프라인을 생성하여 돌려준다."""
    
    vectorizer = TfidfVectorizer(max_features=None, norm=None, smooth_idf=True,
                                 sublinear_tf=True, use_idf=True, ngram_range=(1,2),
                                 tokenizer=str.split)
    clf = MultinomialNB(alpha=0.1)
    pipeline = Pipeline([
        ('vect', vectorizer),
        ('clf', clf),
    ])
    
    return pipeline


def main():
    """영화 리뷰 문서 집합에 대한 긍부정 분석을 수행한다."""
    
    input_file_name = "../data/nsmc/ratings.ma.txt"
    documents, labels = read_documents_with_labels(input_file_name)
    pipe_line = build_pipeline()

    cross_val_set = KFold(n=len(documents), n_folds=NUM_FOLDS, shuffle=True)
    accuracies = []

    for train, test in cross_val_set:
        train_documents = documents[train]
        train_labels = labels[train]
        test_documents = documents[test]
        test_labels = labels[test]

        pipe_line.fit(train_documents, train_labels)
        pred_labels = pipe_line.predict(test_documents)

        accuracy = accuracy_score(test_labels, pred_labels)
        accuracies.append(accuracy)

    print("Avg Accuracy: {}, Std Dev: {}".format(np.mean(accuracies),
                                                 np.std(accuracies)))


# 실행
main()

Avg Accuracy: 0.793195, Std Dev: 0.003378549984830775


5절 교차 검증으로 측정한 정확도는 80% 정도이다. 이보다 정확도를 높이기 위해서는 여러 가지 실험이 요청된다. 자질로 사용하는 품사의 선택, 빈도 범위의 제한 등을 생각해 볼 수 있다. 

# Doc2Vec을 응용한 감성 분석
## Doc2Vec
우리는 앞서 하나의 문서를 출현 어휘의 빈도(TF), 또는 TF-IDF로 표현하는 어휘 벡터로 표현하고 여러 개의 문서를 원소로 가진 문서 집합을 문서-어휘 행렬로 표현하여 군집과 분류 과제에 적용하였다. Doc2Vec은 Word2Vec의 차원 축소 방법을 문서-어휘 행렬의 차원 축소에 적용하여 하나의 문서를 하나의 단어처럼 처리하는 것이다. 즉, 한 문서의 ID를 단어로 그 문서를 나타내는 어휘 벡터를 문맥 단어로 보고 임베딩을 구현한 것이다. 그렇게 하면 차원이 훨씬 작아진 공간에 문서 집합을 표현하여 처리가 가능하다.

![Doc2Vec의 도식](figs/d2v.png)

In [11]:
import ujson
import numpy as np
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

FEATURE_POSES = ["NC", "NQ", "PV", "PA", "MM", "MA", "II", "S"]
PRED_POSES = ["PV", "PA"]
INPUT_FILE_NAME = "../data/nsmc/ratings.ma.txt"
MA_KEY = "document_ma"
LABEL_KEY = "label"
NUM_FOLDS = 5


def read_documents_with_labels(input_file_name):
    """주어진 이름을 파일에서 문서와 레이블들을 읽어서 돌려준다."""
    
    print_log_mesg("Reading input documents.")
    documents = []
    labels = []

    with open(input_file_name, "r", encoding="utf-8") as input_file:
        for line in input_file:
            json_obj = ujson.loads(line)
            label = json_obj[LABEL_KEY]
            labels.append(label)
            morphs = []

            for sent_anal in json_obj[MA_KEY]:
                for morph_lex, morph_cat in sent_anal:
                    if morph_cat not in FEATURE_POSES:
                        continue
                        
                    if morph_cat in PRED_POSES:
                        morph_lex += "다"
                        
                    morphs.append(morph_lex)

            # document = " ".join(morphs)
            # documents.append(document)
            documents.append(morphs)

    return documents, labels


def get_tagged_documents(documents, labels):
    """주어진 문서 집합과 레이블 집합에서 태그 문서 집합을 생성하여 돌려준다."""
    
    print_log_mesg("Getting tagged documents.")
    tagged_documents = [TaggedDocument(d, [l]) 
                        for d, l in zip(documents, labels)]
    
    return tagged_documents


def train_vectorizer(tagged_train_documents):
    """벡터라이저를 생성하여 학습한 뒤 돌려준다."""
    
    print_log_mesg("Training vectorizer.")
    vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
    vectorizer.build_vocab(tagged_train_documents)
    
    for epoch in range(10):
        print_log_mesg("Epoch: {}".format(epoch))
        vectorizer.train(tagged_train_documents)
        vectorizer.alpha -= 0.002  # decrease the learning rate
        vectorizer.min_alpha = vectorizer.alpha  # fix the learning rate, no decay
    
    return vectorizer


def get_doc_vec_mat(vectorizer, tagged_documents):
    """주어진 태그 문서 집합으로부터 문서-벡터 행렬을 생성하여 돌려준다."""
    
    print("Building document vectorizer matrix.")
    doc_vec_mat = [vectorizer.infer_vector(d.words) for d in tagged_documents]
    
    return doc_vec_mat


def build_classifier(train_doc_vec_mat, train_labels):
    """분류자를 생성하여 돌려준다."""
    
    print_log_mesg("Building classifier.")
    clf = LogisticRegression(random_state=1234)
    clf.fit(train_doc_vec_mat, train_labels)
    
    return clf


def test_classifier(clf, test_doc_vec_mat, test_labels):
    """분류자를 시험한다."""
    
    print_log_mesg("Testing classifier.")
    pred_labels = clf.predict(test_doc_vec_mat)
    accuracy = accuracy_score(test_labels, pred_labels)

    return accuracy    


def print_log_mesg(mesg):
    """로그 메시지를 출력한다."""
    
    print(mesg, flush=True)


def main():
    input_file_name = "../data/nsmc/ratings.ma.txt"
    documents, labels = read_documents_with_labels(input_file_name)
    train_documents, test_documents, train_labels, test_labels = train_test_split(documents, labels)    
    tagged_train_documents = get_tagged_documents(train_documents, train_labels)
    tagged_test_documents = get_tagged_documents(test_documents, test_labels)
    vectorizer = train_vectorizer(tagged_train_documents)
    train_doc_vec_mat = get_doc_vec_mat(vectorizer, tagged_train_documents)
    test_doc_vec_mat = get_doc_vec_mat(vectorizer, tagged_test_documents)
    clf = build_classifier(train_doc_vec_mat, train_labels)
    accuracy = test_classifier(clf, test_doc_vec_mat, test_labels)
    print("Accuracy: {}".format(accuracy))   
    
    
main()

Reading input documents.
Getting tagged documents.
Getting tagged documents.
Training vectorizer.
Epoch: 0


ValueError: You must specify either total_examples or total_words, for proper job parameters updationand progress calculations. The usual value is total_examples=model.corpus_count.

# 차별어 분석
한편 긍정 리뷰와 부정 리뷰에 사용되는 차별적인 형태소를 살펴봄으로써 감성어를 유추해 볼 수도 있다. 차별어 분석에는 여러 가지 방법을 사용할 수 있는데, 여기서는 다음과 같이 정의되는 KL 발산(Kullback-Leibler divergence)을 활용한다.

$$
    D_A(q_A||r)=q_A(W)\cdot\log\frac{q_A(W)}{r(W)}
$$

>김하수, 손현정, 이재윤, 강범일 (2013). 정치와 언어의 관계에 대한 양적 분석 시론. 『담화와 인지』, 20(1), 79-111.

위의 식은 특정 형태소 $W$가 특정 문서 집합 $A$에서의 출현 확률 $q_A(W)$와 모든 문서 집합에서의 출현 확률 $r(W)$와의 평균과의 차이를 구하는 나타내는 다이버전스 값을 구하는 식이다. 예를 들어, 형태소 `몰입`이 긍정 리뷰 문서 집합에 30번, 부정 리뷰 문서 집합에 20번 사용되었는데, 두 문서 집합의 크기(총 형태소 발현 수)가 각각 1000과 2000이라면 형태소 `몰입`의 긍정 문서 집합에서의 KL 다이버전스는 다음과 같이 구한다.

$$
D_{긍정}(q_{긍정}||r) = 0.03 \times \log \frac{0.03}{0.02} = 0.01212
$$

위에서 형태소 `몰입`의 긍정과 부정 문서 집합에서의 출현 확률, 즉 $q_{긍정}(몰입)$과 $q_{부정}(몰입)$은 다음과 같이 구한 것이다.

$$
    q_{긍정}(몰입) = \frac{30}{1000} = 0.03\\
    q_{부정}(몰입) = \frac{20}{2000} = 0.01
$$

$r(몰입)$은 형태소 `몰입`의 두 출현 확률의 산술 평균이므로 다음과 같이 간단히 구할 수 있다.

$$
    r(몰입) = \frac{0.03 + 0.01}{2} = 0.02
$$

위와 같은 계산을 두 문서 집합에 출현한 모든 형태소에 대해 수행하면 각 문서 집합별로 차별성의 순위과 매겨진 차별어의 목록을 얻을 수 있다. 다음은 위의 과정을 구현한 것이다.

In [9]:
import math
import ujson
from collections import Counter

INPUT_FILE_NAME = "../data/nsmc/ratings.ma.txt"
OUTPUT_FILE_NAME = "../data/nsmc/ratings.kl.txt"
MA_KEY = "document_ma"
LABEL_KEY = "label"
PRED_POSES = ["PV", "PA"]


def get_morph_counters(input_file):
    """주어진 입력 파일에서 긍부정 리뷰별 형태소 빈도를 계수하여 돌려준다."""
    
    pos_morph_counter = Counter()
    neg_morph_counter = Counter()

    for line in input_file:
        json_obj = ujson.loads(line.strip())

        for sent_ma in json_obj[MA_KEY]:
            for morph_lex, morph_cat in sent_ma:
                if morph_cat in PRED_POSES:
                    morph_lex += "다"
                    
                if json_obj[LABEL_KEY] == "1":
                    pos_morph_counter[morph_lex] += 1
                else:
                    neg_morph_counter[morph_lex] += 1

    return pos_morph_counter, neg_morph_counter


def get_morph_probs(morph_counter):
    """형태소 빈도에서 형태소 출현 확률을 구하여 돌려준다."""
    
    morph_probs = Counter()
    sum_morph_freqs = sum(morph_counter.values())

    for morph, freq in morph_counter.items():
        morph_probs[morph] = freq / sum_morph_freqs

    return morph_probs


def get_kl_divs(morph_probs_a, morph_probs_b):
    """주어진 형태소 출현 빈도들로부터 KL 발산을 구하여 돌려준다."""
    
    kl_divs_a = Counter()
    kl_divs_b = Counter()
    all_morphs = set(morph_probs_a.keys()) | set(morph_probs_b.keys())

    for morph in all_morphs:
        morph_prob_a = morph_probs_a[morph]
        morph_prob_b = morph_probs_b[morph]
        avg_morph_prob = (morph_prob_a + morph_prob_b) / 2

        # 형태소 발현 확률이 0이면 KL 발산을 음의 무한대로 정의한다.

        if morph_prob_a == 0.0:
            kl_divs_a[morph] = -math.inf
        else:
            kl_divs_a[morph] = morph_prob_a * math.log(morph_prob_a /
                                                       avg_morph_prob)

        if morph_prob_b == 0.0:
            kl_divs_b[morph] = -math.inf
        else:
            kl_divs_b[morph] = morph_prob_b * math.log(morph_prob_b /
                                                       avg_morph_prob)

    return kl_divs_a, kl_divs_b


def write_kl_divs(output_file, kl_divs_a, kl_divs_b):
    """긍부정 리뷰 차별어를 출력 파일에 기록한다."""
    
    print("긍정 리뷰 차별어\t\t부정 리뷰 차별어", file=output_file)

    for (morph_a, kl_div_a), (morph_b, kl_div_b) \
            in zip(kl_divs_a.most_common(1000), kl_divs_b.most_common(1000)):
        print("{}\t{}\t{}\t{}".format(morph_a, kl_div_a, morph_b, kl_div_b),
              file=output_file)

        
def main():
    """영화 리뷰 문서 집합에 대한 차별어 분석을 수행한다."""
    
    input_file_name = "../data/nsmc/ratings.ma.txt"
    output_file_name = "../data/nsmc/ratings.kl.txt"
    
    with open(output_file_name, "w", encoding="utf-8") as output_file, \
            open(input_file_name, "r", encoding="utf-8") as input_file:
        pos_morph_counter, neg_morph_counter = get_morph_counters(input_file)
        pos_morph_probs = get_morph_probs(pos_morph_counter)
        neg_morph_probs = get_morph_probs(neg_morph_counter)
        pos_kl_divs, neg_kl_divs = get_kl_divs(pos_morph_probs, neg_morph_probs)
        write_kl_divs(output_file, pos_kl_divs, neg_kl_divs)


# 실행
main()

위의 스크립트를 실행하여 얻은 결과의 일부를 보이면 다음과 같다.

```
긍정 리뷰 차별어                부정 리뷰 차별어
의	0.0030305479398032554	...	0.0026578163122276334
최고	0.0025301699948598174	?	0.0022992807452018958
!	0.002289848602412086	없다	0.0018586433956991094
보	0.00213792165990402	없	0.0018352988507996757
좋다	0.0020780186501243706	이	0.0018144233213667111
아	0.0020493147000882786	재미	0.0016330268229068834
있	0.0017897826562821816	..	0.0015402829340524502
요	0.0017810179437989693	도	0.001460325558317762
ㄴ	0.0017277690504627478	아깝다	0.0013598024720769494
ㅂ니다	0.0014351697366526658	.	0.0013163588217444714
정말	0.0013578677947521263	가	0.0011874300543378757
을	0.0013067637616118821	고	0.0011075771663953057
재미있다	0.0012387389829662816	쓰레기	0.0010802053402851498
!!	0.0011973711060547345	만	0.0010782590295901468
영화	0.0011942607454055126	냐	0.0009843735393425122
게	0.001192402756169758	최악	0.0009733389938440608
었	0.001011066863468637	뭐	0.0009691194604396059
~	0.0009836627184739587	지루	0.000846378461091694
^^	0.0008974384370681538	로	0.0008411965388897139
```

# 기분 분석
이번에는 한국 가요 가사의 장르별 기분 분석을 시도해 보자. 앞서 서술한 바와 같이 이 과제를 수행하기 위해서는 감성어 목록이 주어져야 한다. 한국어의 경우에는 아직 검증되었으며 공개된 감성어 목록이 존재하지 않는다. 우리는 실험을 위해 심리학에서 연구된 결과를 사용하기로 한다.

>최근 공개된 감성어 목록 작성 프로젝트가 시작되었다. <http://openhangul.com> 참조.

>손선주, 박미숙, 박지은, 손진훈 (2012) 한국어 감정표현단어의 추출과 범주화, 『감성과학』 15(1): 106-120.

발표 논문에서 추출한 감성어 목록은 다음과 같은 형식으로 되어 있다.

```
1   가련하다    슬픔  88.8    5.62    2.36
1   가뿐하다    기쁨  81.3    5.36    2.89
2   가슴 아프다  슬픔  93.8    7.62    2.14
2   감개무량하다  기쁨  83.8    7.23    2.55
3   가슴앓이    슬픔  90  7.75    1.98
3   감격하다    기쁨  92.5    7.81    2.16
4   가엾다 슬픔  87.5    6.26    2.59
4   감동하다    기쁨  88.8    7.48    2.32
5   각박하다    슬픔  42.5    5.87    2.47
5   감미롭다    기쁨  77.5    6.03    2.63
6   간절하다    슬픔  57.5    4.95    2.72
6   감복하다    기쁨  61.3    6.64    2.65
7   걱정하다    슬픔  66.3    4.55    2.29
7   감사하다    기쁨  97.5    7.24    2.26
8   고달프다    슬픔  77.5    5.66    2.79
8   감회  기쁨  51.2    4.58    2.5
9   고독하다    슬픔  70  6.29    2.68
9   감흥  기쁨  61.3    5.83    2.59
10  곤욕스럽다   슬픔  32.5    4.62    2.71
10  경쾌하다    기쁨  95  7.42    2.53
```

이 자료는 해당 감성어의 상대 빈도, 설문 조사에 의해 파악된 세부 감성, 강도의 평균과 표준 편차가 주어져 있다. 우리는 감성어와 세부 감성만 이용하며, 감성어가 구로 주어진 경우는 생략하였다.

In [10]:
import os.path
from collections import Counter
from collections import defaultdict
import ujson

SENT_WORD_FILE_NAME = "../data/sohn/sent_words.txt"
INPUT_FILE_NAME = "../data/kpop/kpop_1990-2015.ma.txt"
OUTPUT_FILE_NAME = "../data/kpop/kpop_2014-2015.sa.txt"
PRED_POSES = ["PV", "PA"]
TARGET_YEARS = ["2014", "2015"]
MA_KEY = "lyrics_ma"
GENRE_KEY = "song_genre"
DATE_KEY = "distribution_date"

                
def read_sent_words():
    """감성어 사전을 파일에서 읽어서 돌려준다."""
    
    sent_words = {}

    with open(SENT_WORD_FILE_NAME, "r", encoding="utf-8") as sent_word_file:
        for line in sent_word_file:
            elems = line.strip().split("\t")
            word = elems[1]
            sent = elems[2]

            if " " in word:
                continue

            sent_words[word] = sent

    return sent_words


def get_genre_sent_word_counters(input_file_name, sent_words):
    """장르별 감성어 빈도를 계수하여 돌려준다."""
    
    genre_sent_word_counters = defaultdict(Counter)

    with open(input_file_name, "r", encoding="utf-8") as input_file:
        for line in input_file:
            json_obj = ujson.loads(line.strip())
            dist_date = json_obj[DATE_KEY]
            dist_year = dist_date[:4]
            
            if dist_year not in TARGET_YEARS:
                continue

            for sent_ma in json_obj[MA_KEY]:
                for morph_lex, morph_cat in sent_ma:
                    if morph_cat in PRED_POSES:
                        morph_lex += "다"

                    if morph_lex not in sent_words:
                        continue

                    genre = json_obj[GENRE_KEY]
                    genre_sent_word_counters[genre][morph_lex] += 1

    return genre_sent_word_counters


def get_genre_mood_counters(genre_sent_word_counters, sent_words):
    """장르별 감성어 빈도에서 장르별 기분 빈도를 생성하여 돌려준다."""
    
    genre_mood_counters = defaultdict(Counter) 
    
    for genre, sent_word_counter in genre_sent_word_counters.items():
        for sent_word, count in sent_word_counter.items():
            mood = sent_words[sent_word]
            genre_mood_counters[genre][mood] += count
            
    return genre_mood_counters
    
    
def write_genre_mood_dist(output_file_name, genre_mood_counters, sent_words):
    """장르별 기분의 분포를 파일에 기록한다."""
    
    mood_names = sorted(set(sent_words.values()))
    
    with open(output_file_name, "w", encoding="utf-8") as output_file:
        print(u"장르\t{}".format("\t".join(mood_names)), file=output_file)

        for genre, mood_counter in genre_mood_counters.items():
            mood_counts = []
            mood_sum = 0

            for mood_name in mood_names:
                mood_count = mood_counter[mood_name]
                mood_counts.append(mood_count)
                mood_sum += mood_count

            mood_props = []

            for mood_count in mood_counts:
                mood_prop = str(mood_count / mood_sum)
                mood_props.append(mood_prop)

            print("{}\t{}".format(genre, "\t".join(mood_props)), file=output_file)

                
def main():
    """한국 가요 가사에 대한 기분 분석을 수행한다."""
    
    input_file_name = "../data/kpop/kpop_1990-2015.ma.txt"
    output_file_name = "../data/kpop/kpop_2014-2015.sa.txt"
    sent_words = read_sent_words()
    genre_sent_word_counters = \
        get_genre_sent_word_counters(input_file_name,  sent_words)
    genre_mood_counters = \
        get_genre_mood_counters(genre_sent_word_counters, sent_words)
    write_genre_mood_dist(output_file_name, genre_mood_counters, sent_words)


# 실행
main()

위 스크립트의 실행 결과를 마이크로소프트 엑셀 등의 스프레드 시트에를 이용해 시각화하면 다음과 같은 장르별 기분 분포도를 얻을 수 있다.

![2014-2015 한국 가요 가사 기분 분석](figs/kpop-mood.png)